In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This command unzips your data.
# -q means "quiet" (so it doesn't print all 3000 filenames)
# -d /content/ means "destination is the /content/ folder"

!unzip -q "/content/drive/My Drive/dataset.zip" -d /content/

In [7]:
import argparse
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from pathlib import Path
from tqdm import tqdm
import random
import json


# === 1. REPRODUCIBILITY ===
def set_seed(seed=42):
    """Sets the seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # Makes CUDA operations deterministic, but can be a bit slower
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


# === 2. THE MODEL (SignLanguageGRU) ===

class Attention(nn.Module):
    """Simple dot-product attention."""

    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.weights = nn.Parameter(torch.randn(hidden_size))

    def forward(self, x):
        # x shape: (batch_size, seq_len, hidden_size)
        scores = torch.matmul(x, self.weights)  # (batch_size, seq_len)
        attn_weights = F.softmax(scores, dim=1)  # (batch_size, seq_len)
        context_vector = torch.sum(x * attn_weights.unsqueeze(-1), dim=1)
        return context_vector, attn_weights


class SignLanguageGRU(nn.Module):
    def __init__(self, input_size, num_classes, hidden_size=128, num_gru_layers=2, dropout=0.3):
        super(SignLanguageGRU, self).__init__()

        self.gru = nn.GRU(
            input_size,
            hidden_size,
            num_layers=num_gru_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_gru_layers > 1 else 0
        )

        gru_output_size = hidden_size * 2  # Bidirectional
        self.bn1 = nn.BatchNorm1d(gru_output_size)
        self.attention = Attention(gru_output_size)

        self.fc1 = nn.Linear(gru_output_size, 64)
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        gru_out, _ = self.gru(x)
        context_vector, _ = self.attention(gru_out)
        bn_out = self.bn1(context_vector)

        x = self.relu(self.fc1(bn_out))
        x = self.dropout1(x)
        x = self.fc2(x)
        return x


# === 3. DATA LOADER ===

def get_class_mapping(data_root):
    """Finds all class folders and creates a name-to-index mapping."""
    data_path = Path(data_root)
    class_names = sorted([d.name for d in data_path.iterdir() if d.is_dir()])
    if not class_names:
        raise FileNotFoundError(f"No class subdirectories found in {data_root}")
    class_to_idx = {name: i for i, name in enumerate(class_names)}

    # Save the class mapping
    try:
        np.save('class_mapping.npy', class_to_idx)
        print(f"Saved class mapping to class_mapping.npy")
    except Exception as e:
        print(f"Warning: Could not save class_mapping.npy. Error: {e}")

    return class_to_idx


class SignLanguageDataset(Dataset):
    """
    Custom PyTorch Dataset for loading single, *preprocessed* .npy sequence files.
    """
    def __init__(self, data_root, class_to_idx):
        self.data_root = Path(data_root)
        self.class_to_idx = class_to_idx

        self.filepaths = []
        self.labels = []

        # Load all file paths and labels
        for class_name, label_idx in self.class_to_idx.items():
            class_dir = self.data_root / class_name
            for seq_file in class_dir.glob("*.npy"):
                self.filepaths.append(seq_file)
                self.labels.append(label_idx)

        if not self.filepaths:
            raise FileNotFoundError(f"No .npy files found in {self.data_root} or its subdirectories.")

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        path = self.filepaths[idx]
        label = self.labels[idx]

        # Load the *preprocessed* (20, 450) sequence
        processed_sequence = np.load(path)

        return torch.tensor(processed_sequence, dtype=torch.float32), \
            torch.tensor(label, dtype=torch.long)


# === 4. TRAIN & VALIDATE FUNCTIONS ===

def train_fn(loader, model, optimizer, criterion, device):
    """One epoch of training."""
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    loop = tqdm(loader, desc="Training")
    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        corrects += torch.sum(preds == labels)
        total += labels.size(0)
        loop.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * corrects / total
    return epoch_loss, epoch_acc


def val_fn(loader, model, criterion, device):
    """One epoch of validation."""
    model.eval()
    running_loss = 0.0
    corrects = 0
    total = 0

    loop = tqdm(loader, desc="Validating")
    with torch.no_grad():
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels)
            total += labels.size(0)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * corrects / total
    return epoch_loss, epoch_acc


# === 5. MAIN ORCHESTRATION ===

def main(args):
    set_seed(42)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # --- Data Setup ---
    print("Loading data...")
    class_to_idx = get_class_mapping(args.data_dir)
    num_classes = len(class_to_idx)

    dataset = SignLanguageDataset(
        args.data_dir,
        class_to_idx
    )

    # --- Train/Val Split ---
    val_percent = 0.2
    val_size = int(len(dataset) * val_percent)
    train_size = len(dataset) - val_size

    # Ensure val_size is not zero if dataset is very small
    if val_size == 0 and train_size > 0:
        val_size = 1
        train_size = len(dataset) - val_size

    if train_size == 0 or val_size == 0:
        print(f"Error: Not enough data to create a train/val split. Found {len(dataset)} samples.")
        return

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    print(f"Total samples: {len(dataset)}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Number of classes: {num_classes}")

    # --- DataLoaders ---
    num_workers = 4 if device == "cuda" else 2
    print(f"Using {num_workers} data loader workers.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if device == "cuda" else False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if device == "cuda" else False
    )

    # --- Model, Loss, Optimizer ---
    model = SignLanguageGRU(
        input_size=450,
        num_classes=num_classes,
        hidden_size=args.hidden_size,
        num_gru_layers=args.num_layers,
        dropout=args.dropout
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    # --- Training Loop ---
    best_val_loss = float('inf')

    # Initialize training history for visualization
    training_history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rate': []
    }

    for epoch in range(args.epochs):
        print(f"\n--- Epoch {epoch + 1}/{args.epochs} ---")

        train_loss, train_acc = train_fn(train_loader, model, optimizer, criterion, device)
        val_loss, val_acc = val_fn(val_loader, model, criterion, device)

        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']

        # Save metrics to history for visualization
        training_history['train_loss'].append(float(train_loss))
        training_history['train_acc'].append(float(train_acc))
        training_history['val_loss'].append(float(val_loss))
        training_history['val_acc'].append(float(val_acc))
        training_history['learning_rate'].append(float(current_lr))

        print(f"Epoch {epoch + 1} Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
        print(f"  Learning Rate: {current_lr:.6f}")

        scheduler.step(val_loss)

        # --- Checkpointing ---
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            model_save_path = "best_model.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"  -> New best model saved to {model_save_path} (Val Loss: {best_val_loss:.4f})")

        # Save training history periodically (every 10 epochs and at the end)
        if (epoch + 1) % 10 == 0 or epoch == args.epochs - 1:
            with open('training_history.json', 'w') as f:
                json.dump(training_history, f, indent=2)

    print("\nTraining complete!")
    print(f"Best validation loss: {best_val_loss:.4f}")

    # Save final training history
    with open('training_history.json', 'w') as f:
        json.dump(training_history, f, indent=2)
    print("Training history saved to training_history.json (for visualization)")


# === MODIFIED: Replaced argparse with a hardcoded config class ===
if __name__ == "__main__":

    # Define a simple class to hold our settings
    class Config:
        def __init__(self):
            # --- Paths and Data ---
            # This is the path inside Colab where you unzipped your data
            self.data_dir = "/content/dataset_processed"
            self.seq_len = 20

            # --- Training Params ---
            self.epochs = 150
            self.batch_size = 64  # Bigger batch size is good for GPUs
            self.lr = 1e-3

            # --- Model Params ---
            self.hidden_size = 128
            self.num_layers = 2
            self.dropout = 0.4

    # Create the config object
    args = Config()

    # Run the main function
    main(args)


Using device: cuda
Loading data...
Saved class mapping to class_mapping.npy
Total samples: 3156
Training samples: 2525
Validation samples: 631
Number of classes: 26
Using 4 data loader workers.

--- Epoch 1/150 ---


Validating: 100%|██████████| 10/10 [00:00<00:00, 37.08it/s]


Epoch 1 Summary:
  Train Loss: 2.8401 | Train Acc: 20.99%
  Val Loss:   2.2444 | Val Acc:   36.29%
  Learning Rate: 0.001000
  -> New best model saved to best_model.pth (Val Loss: 2.2444)

--- Epoch 2/150 ---


Validating: 100%|██████████| 10/10 [00:00<00:00, 38.88it/s]


Epoch 2 Summary:
  Train Loss: 1.6575 | Train Acc: 53.58%
  Val Loss:   1.0597 | Val Acc:   73.85%
  Learning Rate: 0.001000
  -> New best model saved to best_model.pth (Val Loss: 1.0597)

--- Epoch 3/150 ---


...

Epoch 148 Summary:
  Train Loss: 0.0044 | Train Acc: 100.00%
  Val Loss:   0.0000 | Val Acc:   100.00%
  Learning Rate: 0.000000

--- Epoch 149/150 ---


Validating: 100%|██████████| 10/10 [00:00<00:00, 21.89it/s]


Epoch 149 Summary:
  Train Loss: 0.0040 | Train Acc: 99.96%
  Val Loss:   0.0000 | Val Acc:   100.00%
  Learning Rate: 0.000000

--- Epoch 150/150 ---


Validating: 100%|██████████| 10/10 [00:00<00:00, 23.10it/s]

Epoch 150 Summary:
  Train Loss: 0.0047 | Train Acc: 99.88%
  Val Loss:   0.0000 | Val Acc:   100.00%
  Learning Rate: 0.000000

Training complete!
Best validation loss: 0.0000
Training history saved to training_history.json (for visualization)


In [4]:
import argparse
import os
import numpy as np
from pathlib import Path
from tqdm import tqdm

# --- This is the function you are looking for ---
def preprocess_keypoints(sequence, seq_len):
    """
    Applies normalization and motion deltas to the raw keypoint sequence.
    Input shape: (seq_len, 225)
    Output shape: (seq_len, 450)
    """
    if not isinstance(sequence, np.ndarray):
        sequence = np.array(sequence)

    # --- A. Normalization (Translation Invariance) ---
    # Reshape to (seq_len, num_groups, num_keypoints, num_coords)
    pose = sequence[:, :99].reshape(seq_len, 33, 3)
    lh = sequence[:, 99:162].reshape(seq_len, 21, 3)
    rh = sequence[:, 162:225].reshape(seq_len, 21, 3)

    # Get reference points (wrist for hands, nose for pose)
    nose = pose[:, 0:1, :]
    lwrist = lh[:, 0:1, :]
    rwrist = rh[:, 0:1, :]

    # Subtract the reference point. Zeros (missing) will just subtract zero.
    pose_norm = pose - nose
    lh_norm = lh - lwrist
    rh_norm = rh - rwrist

    # Flatten back to (seq_len, 225)
    normalized_sequence = np.concatenate([
        pose_norm.reshape(seq_len, 99),
        lh_norm.reshape(seq_len, 63),
        rh_norm.reshape(seq_len, 63)
    ], axis=1)

    # --- B. Motion Deltas (Velocity) ---
    # Calculate frame-to-frame differences
    deltas = np.diff(normalized_sequence, axis=0)

    # Pad the first frame with zeros
    deltas = np.concatenate([np.zeros((1, 225)), deltas], axis=0)

    # --- C. Concatenate Features ---
    # Final shape: (seq_len, 450)
    final_features = np.concatenate([normalized_sequence, deltas], axis=1)

    return final_features

def main(args):
    """
    Processes all .npy files from the input directory, applies preprocessing,
    and saves them to the output directory, maintaining the class structure.
    """
    input_dir = Path(args.data_dir)
    output_dir = Path(args.output_dir)
    seq_len = args.seq_len

    print(f"Starting preprocessing...")
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")

    # Find all class names
    class_names = sorted([d.name for d in input_dir.iterdir() if d.is_dir()])
    if not class_names:
        print(f"Error: No class subdirectories found in {input_dir}")
        return

    print(f"Found {len(class_names)} classes.")

    total_files = 0
    # Loop over each class
    for class_name in tqdm(class_names, desc="Processing Classes"):
        input_class_dir = input_dir / class_name
        output_class_dir = output_dir / class_name

        # Create the output class directory
        output_class_dir.mkdir(parents=True, exist_ok=True)

        # Find all .npy files
        filepaths = list(input_class_dir.glob("*.npy"))
        if not filepaths:
            print(f"Warning: No .npy files found in {input_class_dir}")
            continue

        # Process each file
        for filepath in tqdm(filepaths, desc=f"Class {class_name}", leave=False):
            try:
                # Load the raw (20, 225) sequence
                raw_sequence = np.load(filepath)

                # Check shape
                if raw_sequence.shape != (seq_len, 225):
                    print(f"Warning: Skipping {filepath}. Expected shape ({seq_len}, 225), got {raw_sequence.shape}")
                    continue

                # Apply preprocessing (Normalization + Deltas)
                processed_sequence = preprocess_keypoints(raw_sequence, seq_len)

                # Define the new save path
                output_filepath = output_class_dir / filepath.name

                # Save the processed (20, 450) sequence
                np.save(output_filepath, processed_sequence)
                total_files += 1

            except Exception as e:
                print(f"Error processing {filepath}: {e}")

    print(f"\nPreprocessing complete!")
    print(f"Processed and saved {total_files} files to {output_dir}")

# === MODIFIED: Replaced argparse with a hardcoded config class ===
if __name__ == "__main__":

    # Define a simple class to hold our settings
    class Config:
        def __init__(self):
            # --- Paths and Data ---
            # This is the path to your UNZIPPED RAW data (from dataset2.zip)
            self.data_dir = "/content/dataset"

            # This is where the NEW processed files will be saved
            self.output_dir = "/content/dataset_processed"

            self.seq_len = 20

    # Create the config object
    args = Config()

    # Run the main function
    main(args)



Starting preprocessing...
Input directory: /content/dataset
Output directory: /content/dataset_processed
Found 26 classes.


Class space:   0%|          | 0/120 [00:00<?, ?it/s]
                                                    
Class դու:   0%|          | 0/120 [00:00<?, ?it/s]
                                                  
Class երբ:   0%|          | 0/120 [00:00<?, ?it/s]
                                                  
Class ինչ:   0%|          | 0/120 [00:00<?, ?it/s]
                                                  
Class ինքդ:   0%|          | 0/120 [00:00<?, ?it/s]
                                                   
Class հազվագյուտ:   0%|          | 0/120 [00:00<?, ?it/s]
                                                         
Class մենք:   0%|          | 0/132 [00:00<?, ?it/s]
                                                   
Class նա:   0%|          | 0/120 [00:00<?, ?it/s]
                                                 
Class նրա:   0%|          | 0/120 [00:00<?, ?it/s]
                                                  
Class ոչինչ:   0%|          | 0/120 [00:00<?, ?it/s]
         


Preprocessing complete!
Processed and saved 3156 files to /content/dataset_processed
